# FoodVision101: Image Classification

Classifying images of food (belonging to 101 unique food categories).

## Preparation

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import random
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Activation
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
import datetime

Helper functions

In [2]:
def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback
  

In [3]:
def preprocess_image(image,label,img_size=224):
  
  # resize image
  image = tf.image.resize(image,(img_size,img_size))

  # recast as float32
  return tf.cast(image,dtype=tf.float32),label


Set up callbacks

In [11]:
# Create a ModelCheckpoint callback to save model's progress
checkpoint_loc = "model_EfficientNet_checkpoints/model_EfficientNetB0"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_loc,
                                                      monitor="val_acc",
                                                      save_best_only=True,
                                                      verbose=0)

In [5]:
earlystopping_loc = "model_EfficientNet_earlystop/model_EfficientNetB0"
earlystopping_callback_fine_tuning = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                                      min_delta=0,
                                                                      patience=3,
                                                                      restore_best_weights=True,
                                                                      mode="min",
                                                                      verbose=0)

## Execution

Retrieve data

In [6]:
# use tfds.load to get train and test data
(train_data,test_data),dataset_info = tfds.load(name="food101",
                                             split=["train","validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

class_names = dataset_info.features["label"].names

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incomplete9QPICT/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incomplete9QPICT/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


Create train and test sets

In [7]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_image,
                            num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(map_func=preprocess_image,
                          num_parallel_calls=tf.data.AUTOTUNE)

# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

Using EfficientNetB0 as a base model (transfer learning)


In [8]:
# Create base model
base_model = EfficientNetB0(include_top=False)
base_model.trainable = False

16705208/16705208 [==============================] - 0s 0us/step


Design, compile, train, and evaluate an untrainable EfficientNetB0-based model

In [9]:
# Create Functional model 
inputs = Input(shape=(224,224,3),name="input_layer")
x = base_model(inputs,training=False)
x = GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
x = Dense(len(class_names),name="dense_layer")(x)

# Separate activation of output layer so we can output float32 activations
outputs = Activation("softmax",dtype=tf.float32,name="output_layer")(x)
model_EfficientNet = tf.keras.Model(inputs,outputs,name="model_EfficientNet")

# Compile the model
model_EfficientNet.compile(loss="sparse_categorical_crossentropy",
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=["accuracy"])

model_EfficientNet.summary()

Model: "model_EfficientNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense_layer (Dense)         (None, 101)               129381    
                                                                 
 output_layer (Activation)   (None, 101)               0         
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
________________________________

In [13]:
save_path = "model_experiments"

# Fit the feature extraction model for 3 epochs with tensorboard and model checkpoint callbacks
history_EfficientNetB0_no_fine_tuning = model_EfficientNet.fit(train_data,
                                        epochs=1,
                                        steps_per_epoch=len(train_data),
                                        validation_data=test_data,
                                        validation_steps=int(0.15*len(test_data)),
                                        callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path,"model_EfficientNetB0_no_fine_tuning"),
                                                                                      verbose=0,
                                                                                      monitor="val_loss",
                                                                                      save_best_only=True),
                                                   create_tensorboard_callback(dir_name="model_EfficientNet_tensorboard",
                                                                               experiment_name="model_EfficientNetB0_no_fine_tuning")])
# Restore the best weights
model_EfficientNet = tf.keras.models.load_model("model_experiments/model_EfficientNetB0_no_fine_tuning/")

# Check loaded model performance (this should be the same as results_feature_extract_model)
result_EfficientNetB0_no_fine_tuning = model_EfficientNet.evaluate(test_data)
result_EfficientNetB0_no_fine_tuning

Saving TensorBoard log files to: model_EfficientNet_tensorboard/model_EfficientNetB0_no_fine_tuning/20230302-052922
2367/2368 [============================>.] - ETA: 0s - loss: 0.8894 - accuracy: 0.7647

TypeError: ignored

Fine-tuning the EfficientNetB0-based model

In [ ]:
# Set all of the layers' .trainable variable to True (so they're unfrozen)
model_EfficientNet.layers[1].trainable = True

# Recompile the model, using the Adam optimizer with a 10x-lower-than-default learning rate
model_EfficientNet.compile(loss="spare_categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                           metrics=["accuracy"])

# Fine-tune on all layers
history_EfficientNetB0_fine_tuning = model_EfficientNet.fit(train_data,
                                                            epochs=103,
                                                            validation_data=test_data,
                                                            validation_steps=int(0.15*len(test_data)),
                                                            initial_epoch=3,
                                                            callbacks=[earlystopping_callback_fine_tuning,
                                                                       tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path,"model_EfficientNetB0_fine_tuning"),
                                                                                      verbose=0,
                                                                                      monitor="val_loss",
                                                                                      save_best_only=True),
                                                   create_tensorboard_callback(dir_name="model_EfficientNet_tensorboard",
                                                                               experiment_name="model_EfficientNetB0_fine_tuning")])
# Restore the best weights
# model_EfficientNet = tf.keras.models.load_model(checkpoint_loc)
model_EfficientNet = tf.keras.models.load_model("model_experiments/model_EfficientNetB0_fine_tuning/")


# Check loaded model performance (this should be the same as results_feature_extract_model)
result_EfficientNetB0_fine_tuning = model_EfficientNet.evaluate(test_data)
result_EfficientNetB0_fine_tuning

In [ ]:
# need the predicted classes for everything
pred_probs = model_EfficientNet.predict(test_data)
pred_classes = pred_probs.argmax(axis=1) # indices of classes
max_probabilities = pred_probs.max(axis=1)

Plot a confusion matrix

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confusion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### changed (plot x-labels vertically) ###
  plt.xticks(rotation=70,fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")
    

In [ ]:
sklearn_accuracy = accuracy_score(y_labels,pred_classes)
sklearn_accuracy

In [ ]:
confusion_matrix(y_labels,pred_classes,classes=class_names,figsize=(20,20),text_size=20)

View the most-wrong predictions

In [ ]:
# Find the model's most wrong predictions (those with the highest prediction probability but the wrong prediction)
# to find the most wrong predictions, create pandas dataframe
# make additional columns for equality truth, etc.
dataframe = pd.DataFrame(data={"images":image_collection,
                               "labels":y_labels,
                               "pred_labels":pred_classes,
                               "ground_truth":[class_names[i] for i in y_labels],
                               "predictions":[class_names[i] for i in pred_classes],
                               "pred_probability":max_probabilities})
dataframe["pred_correct"] = dataframe["labels"] == dataframe["pred_labels"]

descending_wrong = dataframe[dataframe["pred_correct"]==False].sort_values("pred_probability",ascending=False)
descending_wrong.head(10)